In [1]:
import glob
import pandas as pd
import geopandas as gpd
from itertools import product
from tqdm import tqdm
from custom_functions import *

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 50)

#### 1. Загружаем все файлы
Так же добавим к ним технические колонки связанные с проекцией и будущим расчетом новых признаков

In [3]:
# Train/Test
train = pd.read_csv('train_test/train.csv', index_col=0)
test = pd.read_csv('train_test/test.csv', index_col=0)
concat = pd.concat([train, test]).rename(columns={'long': 'lon'}).reset_index(drop=True)
geometry = gpd.points_from_xy(concat['lon'], concat['lat'])

# Tuning Geoframe
data = gpd.GeoDataFrame(concat, geometry=geometry, crs='EPSG:4326')
# пока дропнем строки с пропусками координат
data.dropna(subset=['lon', 'lat'], inplace=True)

data['utm'] = data.apply(lambda x: find_epsg(x.lon, x.lat), axis=1)


In [4]:
# Infrastructure files
files_path = glob.glob('fed_districts/*.csv')
files = [pd.read_csv(f, index_col=0) for f in files_path]
concat_files = pd.concat(files).reset_index(drop=True)
geometry_files = gpd.points_from_xy(concat_files['lon'], concat_files['lat'])

# Tuning Geoframe
infrastructure = gpd.GeoDataFrame(
    concat_files, geometry=geometry_files, crs='EPSG:4326'
)

infrastructure['utm'] = infrastructure.apply(lambda x: find_epsg(x.lon, x.lat), axis=1)

#### 2. Обогащаем данные 
Итак, у нас 2 основных задачи: поиск ближайшего к банкомату объекта инфрастуркутры и поиск количества точек инфраструктуры в радиусе 250м. Для корректности вычисляемых величин, нам нужно учесть географическое расположение объектов. Для решения данной задачи воспользуемся разделением объектов (как банкоматов, так и точек инфраструктуры) на UTM зоны.

In [5]:
# Вытаскиваем массив UTM зон встречающихся в наших данных
utm_zones = data['utm'].unique()
# Вытаскиваем массив типов инфраструктуры
infr_obj = infrastructure['type'].unique()
# Пары зона - объект
utm_obj = list(product(utm_zones, infr_obj))

In [6]:
# Разделяем датафреймы по UTM зоне и записываем в словарь с соответсвующем ключем
utm_data = {}
for utm in utm_zones:
    utm_data[utm] = data[data['utm'] == utm]

utm_infrastructure = {}
for utm, obj in utm_obj:
    mask = (infrastructure['utm'] == utm) & (infrastructure['type'] == obj)
    utm_infrastructure[(utm, obj)] = infrastructure[mask]

In [7]:
# Считаем расстояния до ближайшего объекта инфраструктуры
for utm, obj in tqdm(utm_obj):
    # Имя будущей колонки с дистанцией
    dist_col_name = f'nearest_{obj}'
    # Корректируем проекцию для текущей итерации
    cur_data = utm_data[utm].to_crs(f'EPSG:{utm}')
    cur_obj = utm_infrastructure[(utm, obj)].to_crs(f'EPSG:{utm}')
    # Добавляем колонку с дистанцией до ближайшего объекта инфраструктуры
    dist_to_nearest = (
        cur_data.sjoin_nearest(cur_obj, how='left', distance_col='dist')[['id', 'dist']]
        .rename(columns={'dist': dist_col_name})
        .drop_duplicates(subset='id')
    )
    utm_data[utm] = pd.merge(utm_data[utm], dist_to_nearest, on='id')

100%|█████████████████████████████████████████| 325/325 [00:10<00:00, 30.07it/s]


In [8]:
# Считаем расстояние до ближайшего соседнего банкомата из первичного датасета
for utm in tqdm(utm_zones):
    try:
        utm_data[utm] = dist_to_nearest_neighbour(
            utm_data[utm], epsg=f'EPSG:{utm}'
        ).to_crs('EPSG:4326')
    except:
        pass

100%|███████████████████████████████████████████| 25/25 [00:22<00:00,  1.10it/s]


In [9]:
# Считаем количество объектов в радиусе 500м
for utm, obj in tqdm(utm_obj):
    # Корректируем проекцию для текущей итерации
    cur_data = utm_data[utm].to_crs(f'EPSG:{utm}')
    cur_obj = utm_infrastructure[(utm, obj)].to_crs(f'EPSG:{utm}')
    utm_data[utm] = utm_data[utm].join(points_in_buffer(cur_obj, cur_data, obj, 500))
    utm_data[utm].columns.str.contains

100%|█████████████████████████████████████████| 325/325 [00:10<00:00, 29.76it/s]


In [10]:
enriched_data = pd.concat(utm_data.values())
# Заполняем пропуски в колонках "объекты в радиусе" на ноль
enriched_data.loc[:, enriched_data.columns.str.contains("_in_")] = \
    enriched_data.loc[:, enriched_data.columns.str.contains("_in_")
].fillna(0, inplace=True)

In [11]:
# enriched_data.to_csv('train_test/enriched_data.csv')